<a href="https://colab.research.google.com/github/DENGFUCHEN/web-crawler/blob/master/test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#! -*-coding:utf-8 -*-
from urllib import request, parse
from bs4 import BeautifulSoup as BS
import json
import datetime
import xlsxwriter
starttime = datetime.datetime.now()
url = r'http://www.lagou.com/jobs/positionAjax.json?city=%E5%8C%97%E4%BA%AC'
# 拉鉤網的招聘資訊都是動態獲取的，所以需要通過post來遞交json資訊，預設城市為北京
tag = ['companyName', 'companyShortName', 'positionName', 'education', 'salary', 'financeStage', 'companySize',
'industryField', 'companyLabelList'] # 這是需要抓取的標籤資訊，包括公司名稱，學歷要求，薪資等等
tag_name = ['公司名稱', '公司簡稱', '職位名稱', '所需學歷', '工資', '公司資質', '公司規模', '所屬類別', '公司介紹']
def read_page(url, page_num, keyword): # 模仿瀏覽器post需求資訊，並讀取返回後的頁面資訊
page_headers = {
'Host': 'www.lagou.com',
'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) '
'Chrome/45.0.2454.85 Safari/537.36 115Browser/6.0.3',
'Connection': 'keep-alive'
}
if page_num == 1:
boo = 'true'
else:
boo = 'false'
page_data = parse.urlencode([  # 通過頁面分析，發現瀏覽器提交的FormData包括以下引數
('first', boo),
('pn', page_num),
('kd', keyword)
])
req = request.Request(url, headers=page_headers)
page = request.urlopen(req, data=page_data.encode('utf-8')).read()
page = page.decode('utf-8')
return page
def read_tag(page, tag):
page_json = json.loads(page)
page_json = page_json['content']['result'] # 通過分析獲取的json資訊可知，招聘資訊包含在返回的result當中，其中包含了許多其他引數
page_result = [num for num in range(15)] # 構造一個容量為15的list佔位，用以構造接下來的二維陣列
for i in range(15):
page_result[i] = [] # 構造二維陣列
for page_tag in tag:
page_result[i].append(page_json[i].get(page_tag)) # 遍歷引數，將它們放置在同一個list當中
page_result[i][8] = ','.join(page_result[i][8])
return page_result  # 返回當前頁的招聘資訊
def read_max_page(page): # 獲取當前招聘關鍵詞的最大頁數，大於30的將會被覆蓋，所以最多隻能抓取30頁的招聘資訊
page_json = json.loads(page)
max_page_num = page_json['content']['totalPageCount']
if max_page_num > 30:
max_page_num = 30
return max_page_num
def save_excel(fin_result, tag_name, file_name): # 將抓取到的招聘資訊儲存到excel當中
book = xlsxwriter.Workbook(r'C:\Users\Administrator\Desktop\%s.xls' % file_name) # 預設儲存在桌面上
tmp = book.add_worksheet()
row_num = len(fin_result)
for i in range(1, row_num):
if i == 1:
tag_pos = 'A%s' % i
tmp.write_row(tag_pos, tag_name)
else:
con_pos = 'A%s' % i
content = fin_result[i-1] # -1是因為被表格的表頭所佔
tmp.write_row(con_pos, content)
book.close()
if __name__ == '__main__':
print('**********************************即將進行抓取**********************************')
keyword = input('請輸入您要搜尋的語言型別：')
fin_result = [] # 將每頁的招聘資訊彙總成一個最終的招聘資訊
max_page_num = read_max_page(read_page(url, 1, keyword))
for page_num in range(1, max_page_num):
print('******************************正在下載第%s頁內容*********************************' % page_num)
page = read_page(url, page_num, keyword)
page_result = read_tag(page, tag)
fin_result.extend(page_result)
file_name = input('抓取完成，輸入檔名儲存：')
save_excel(fin_result, tag_name, file_name)
endtime = datetime.datetime.now()
time = (endtime - starttime).seconds
print('總共用時：%s s' % time)